### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/andrewmvd/heart-failure-clinical-data?select=heart_failure_clinical_records_dataset.csv
2. Соберем пайплайн с простейшим препроцессингом
3. Обучим LGBMClassifier и сохраним на диск предобученный пайплайн

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score, roc_curve, get_scorer
from sklearn.metrics import f1_score
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

from lightgbm import LGBMClassifier


Загрузим данные

In [2]:
df = pd.read_csv("./heart_failure_clinical_records_dataset.csv")
df.head(3)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1


In [3]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [4]:
df.isnull().sum()

age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64

In [5]:
df.loc[df['DEATH_EVENT']==1, 'DEATH_EVENT'].count(), df.loc[df['DEATH_EVENT']==0, 'DEATH_EVENT'].count()

(96, 203)

In [6]:
# DEATH EVENT part
df.loc[df['DEATH_EVENT']==1, 'DEATH_EVENT'].count() / df.loc[df['DEATH_EVENT']==0, 'DEATH_EVENT'].count() * 100

47.29064039408867

Разделим данные на train/test и сохраним тестовую выборку на диск (здесь мы ее касаться уже не будем)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = 'DEATH_EVENT'), 
                                                    df['DEATH_EVENT'], test_size=0.33, random_state=12)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [28]:
features_list = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
                                 'ejection_fraction', 'high_blood_pressure', 'platelets',
                                 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']
features_array = list([])

In [31]:
features_array.append(1)

In [32]:
print(features_array)

[1, 1]


In [ ]:
temp = pd.DataFrame()

In [8]:
features = df.drop(columns = 'DEATH_EVENT').columns
cat_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']
num_features = list(set(features) - set(cat_features))
target = 'DEATH_EVENT'

Соберем кусок, ответственный за feature engineering

In [9]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

    

    
#continuous_columns = ["bedrooms", "yr_built", 'sqft_living', 'sqft_lot']

final_transformers = list()

for cat_col in cat_features:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))


for numerical_col in num_features:
    numerical_features_transformer = Pipeline([('selector', NumberSelector(key=numerical_col)),
                                               ('scaler', StandardScaler())
                                              ])
    final_transformers.append((numerical_col, numerical_features_transformer))
    
    
feats = FeatureUnion(final_transformers)

#feature_processing = Pipeline([('feats', feats)])

Добавим простейший классификатор

In [10]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LGBMClassifier(boosting_type='gbdt', max_depth=3, learning_rate=0.1, n_estimators=150, random_state=12, category_features = list(cat_features))),
])

pipeline.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: category_features
CPU times: user 128 ms, sys: 7.52 ms, total: 135 ms
Wall time: 73.4 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('anaemia',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='anaemia')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='anaemia'))])),
                                                ('diabetes',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='diabetes')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='diabetes'))])),
                                                ('high_blood_pressure',
                                                 Pipeline(steps=[('selector'

Посмотрим, как выглядит наш pipeline

In [11]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('anaemia',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='anaemia')),
                                                  ('ohe',
                                                   OHEEncoder(key='anaemia'))])),
                                 ('diabetes',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='diabetes')),
                                                  ('ohe',
                                                   OHEEncoder(key='diabetes'))])),
                                 ('high_blood_pressure',
                                  Pipeline(steps=[('selector',
                                                   FeatureSelector(column='high_blood_pressure...
                                                  ('scaler', StandardScaler())])),
            

Сохраним модель (пайплайн)

In [12]:
with open("LGBMClassifier_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)